In [1]:
%cd /home/jdafflon/GenerativeModels

/mnt_homes/home4T7/jdafflon/GenerativeModels


In [2]:
# Assumption: Make sure that pytorch-fid and piq are installed on the enviroment. So if you haven't install this
# in your environment

In [3]:
from torch.utils.data import DataLoader, Dataset
import torchvision
import numpy as np
import torch
from time import time
import piq


/home/jdafflon/miniconda3/envs/genmodels/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Copied implementation from master
from __future__ import annotations

import torch
from monai.metrics.metric import Metric
from pytorch_fid.fid_score import calculate_frechet_distance
from scipy import linalg

class FIDMetric(Metric):
    """
    Frechet Inception Distance (FID). The FID calculates the distance between two distributions of feature vectors.
    Based on: Heusel M. et al. "Gans trained by a two time-scale update rule converge to a local nash equilibrium."
    https://arxiv.org/abs/1706.08500#. The inputs for this metric should be two groups of feature vectors (with format
    (number images, number of features)) extracted from the a pretrained network.

    Originally, it was proposed to use the activations of the pool_3 layer of an Inception v3 pretrained with Imagenet.
    However, others networks pretrained on medical datasets can be used as well (for example, RadImageNwt for 2D and
    MedicalNet for 3D images). If the chosen model output is not a scalar, usually it is used a global spatial
    average pooling.
    """

    def __init__(self) -> None:
        super().__init__()

    def __call__(self, y_pred: torch.Tensor, y: torch.Tensor, fid_type):
        return get_fid_score(y_pred, y, fid_type)   

    
    
def get_fid_score(y_pred: torch.Tensor, y: torch.Tensor, fid_type) -> torch.Tensor:
    y = y.double()
    y_pred = y_pred.double()

    if y.ndimension() > 2:
        raise ValueError("Inputs should have (number images, number of features) shape.")

    if fid_type not in ['numpy_fid', 'pytorch_fid']:
        mu_y_pred = torch.mean(y_pred, dim=0)
        mu_y = torch.mean(y, dim=0)
    else:
        y = y.cpu().numpy()
        y_pred = y_pred.cpu().numpy()
        mu_y_pred = np.mean(y_pred, axis=0)
        mu_y = np.mean(y, axis=0)

    
    if fid_type == 'fid':
        sigma_y_pred = _cov(y_pred, rowvar=False)
        sigma_y = _cov(y, rowvar=False)
        #sigma_y_pred = torch.cov(y_pred)
        #sigma_y = torch.cov(y)
        return compute_frechet_distance(mu_y_pred, sigma_y_pred, mu_y, sigma_y)
    if fid_type == 'pytorch_fid':
        sigma_y_pred = np.cov(y_pred, rowvar=False)
        sigma_y = np.cov(y, rowvar=False)
        return calculate_frechet_distance(mu_y_pred, sigma_y_pred, mu_y, sigma_y)
    elif fid_type == 'fast_fid':
        sigma_y_pred = _cov_fast_fid(y_pred, rowvar=False)
        sigma_y = _cov_fast_fid(y, rowvar=False)
        return compute_fast_frechet_distance(mu_y_pred, sigma_y_pred, mu_y, sigma_y)
    elif fid_type == 'numpy_fid':
        sigma_y_pred = np.cov(y_pred, rowvar=False)
        sigma_y = np.cov(y, rowvar=False)
        return compute_numpy_frechet_distance(mu_y_pred, sigma_y_pred, mu_y, sigma_y)

def _cov_fast_fid(m: torch.Tensor, rowvar: bool = True) -> torch.Tensor:
    """
    Estimate a covariance matrix of the variables.

    Args:
        m: A 1-D or 2-D array containing multiple variables and observations. Each row of `m` represents a variable,
            and each column a single observation of all those variables.
        rowvar: If rowvar is True (default), then each row represents a variable, with observations in the columns.
            Otherwise, the relationship is transposed: each column represents a variable, while the rows contain
            observations.
    """

    if m.dim() < 2:
        m = m.view(1, -1)

    if not rowvar and m.size(0) != 1:
        m = m.t()

    fact = 1.0 / np.sqrt(m.size(1) - 1)
    ones = torch.ones(1, m.shape[1]).to(m.device).double()
    #ones = torch.ones(m.shape[0], m.shape[1]).to(m.device).double()

    m = m - (torch.mean(m, dim=1, keepdim=True) @ ones)
    return fact * m.squeeze()

def _cov(m: torch.Tensor, rowvar: bool = True) -> torch.Tensor:
    """
    Estimate a covariance matrix of the variables.

    Args:
        m: A 1-D or 2-D array containing multiple variables and observations. Each row of `m` represents a variable,
            and each column a single observation of all those variables.
        rowvar: If rowvar is True (default), then each row represents a variable, with observations in the columns.
            Otherwise, the relationship is transposed: each column represents a variable, while the rows contain
            observations.
    """
    if m.dim() < 2:
        m = m.view(1, -1)

    if not rowvar and m.size(0) != 1:
        m = m.t()

    fact = 1.0 / (m.size(1) - 1)
    m = m - torch.mean(m, dim=1, keepdim=True)
    mt = m.t()
    return fact * m.matmul(mt).squeeze()


def _sqrtm_newton_schulz(matrix: torch.Tensor, num_iters: int = 100) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Square root of matrix using Newton-Schulz Iterative method. Based on:
    https://github.com/msubhransu/matrix-sqrt/blob/master/matrix_sqrt.py. Bechmark shown in:
    https://github.com/photosynthesis-team/piq/issues/190#issuecomment-742039303

    Args:
        matrix: matrix or batch of matrices
        num_iters: Number of iteration of the method

    """
    dim = matrix.size(0)
    norm_of_matrix = matrix.norm(p="fro")
    y_matrix = matrix.div(norm_of_matrix)
    i_matrix = torch.eye(dim, dim, device=matrix.device, dtype=matrix.dtype)
    z_matrix = torch.eye(dim, dim, device=matrix.device, dtype=matrix.dtype)

    s_matrix = torch.empty_like(matrix)
    error = torch.empty(1, device=matrix.device, dtype=matrix.dtype)

    for _ in range(num_iters):
        t = 0.5 * (3.0 * i_matrix - z_matrix.mm(y_matrix))
        y_matrix = y_matrix.mm(t)
        z_matrix = t.mm(z_matrix)

        s_matrix = y_matrix * torch.sqrt(norm_of_matrix)

        norm_of_matrix = torch.norm(matrix)
        error = matrix - torch.mm(s_matrix, s_matrix)
        error = torch.norm(error) / norm_of_matrix

        if torch.isclose(error, torch.tensor([0.0], device=error.device, dtype=error.dtype), atol=1e-5):
            break

    return s_matrix, error


def compute_frechet_distance(
    mu_x: torch.Tensor, sigma_x: torch.Tensor, mu_y: torch.Tensor, sigma_y: torch.Tensor, epsilon: float = 1e-6
) -> torch.Tensor:
    """The Frechet distance between multivariate normal distributions."""
    diff = mu_x - mu_y
    covmean, _ = _sqrtm_newton_schulz(sigma_x.mm(sigma_y))

    # If calculation produces singular product, epsilon is added to diagonal of cov estimates
    if not torch.isfinite(covmean).all():
        offset = torch.eye(sigma_x.size(0), device=mu_x.device, dtype=mu_x.dtype) * epsilon
        covmean, _ = _sqrtm_newton_schulz((sigma_x + offset).mm(sigma_y + offset))

    tr_covmean = torch.trace(covmean)
    return diff.dot(diff) + torch.trace(sigma_x) + torch.trace(sigma_y) - 2 * tr_covmean

def trace_of_matrix_sqrt(C1, C2):
    """
    Computes using the fact that:   eig(A @ B) = eig(B @ A)
    C1, C2    (d, bs) 
    M = C1 @ C1.T @ C2 @ C2.T 
    eig ( C1 @ C1.T @ C2 @ C2.T ) = 
    eig ( C1 @ (C1.T @ C2) @ C2.T ) =      O(d bs^2)
    eig ( C1 @ ((C1.T @ C2) @ C2.T) ) =        O(d bs^2)
    eig ( ((C1.T @ C2) @ C2.T) @ C1 ) =        O(d bs^2)
    eig ( batch_size x batch_size  )      O(bs^3)
    """
    d, bs = C1.shape 
    assert bs <= d, "This algorithm takes O(bs^2d) time instead of O(d^3), so only use it when bs < d.\nGot bs=%i>d=%i. "%(bs, d) # it also computes wrong thing sice it returns bs eigenvalues and there are only d. 
    M = ((C1.t() @ C2) @ C2.t()) @ C1       # computed in O(d bs^2) time.    O(d^^3)
    S = torch.svd( M , compute_uv=True)[1]  # need 'uv' for backprop.
    S = torch.topk(S, bs-1)[0]              # covariance matrix has rank bs-1
    return torch.sum(torch.sqrt(S))

def compute_trace(S):
    tr = torch.sum(torch.norm(S, dim=0)**2)
    return tr

def compute_fast_frechet_distance(
    mu_x: torch.Tensor, sigma_x: torch.Tensor, mu_y: torch.Tensor, sigma_y: torch.Tensor, epsilon: float = 1e-6
) -> torch.Tensor:
    """The Frechet distance between multivariate normal distributions."""
    diff = mu_x - mu_y
    tr_covmean = trace_of_matrix_sqrt(sigma_x, sigma_y) 
    
    #return diff.dot(diff) + torch.trace(sigma_x) + torch.trace(sigma_y) - 2 * tr_covmean
    return diff.dot(diff) + compute_trace(sigma_x) + compute_trace(sigma_y) - 2 * tr_covmean

def compute_numpy_frechet_distance(
    mu_x: np.ndarray, sigma_x: np.ndarray, mu_y: np.ndarray, sigma_y: np.ndarray, epsilon: float = 1e-6
) -> np.float:
    """The Frechet distance between multivariate normal distributions.
    This implementation is based on https://github.com/mseitzer/pytorch-fid/blob/0a754fb8e66021700478fd365b79c2eaa316e31b/src/pytorch_fid/fid_score.py
     """

    mu_x = np.atleast_1d(mu_x)
    mu_y = np.atleast_1d(mu_y)

    sigma_x = np.atleast_2d(sigma_x)
    sigma_y = np.atleast_2d(sigma_y)

    assert mu_x.shape == mu_y.shape, \
        'Synthetic and real mean vectors have different lengths'
    assert sigma_x.shape == sigma_y.shape, \
        'Synthetic and real covariances have different dimensions'

    diff = mu_x - mu_y

    # Product migth be almost singular
    covmean, _ = linalg.sqrtm(sigma_x.dot(sigma_y), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % epsilon
        print(msg)
        offset = np.eye(sigma_x.shape[0]) * epsilon
        covmean = np.linalg.sqrtm((sigma_x + offset).dot(sigma_y + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError(f'Imaginary component {m}')
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma_x)
            + np.trace(sigma_y) - 2 * tr_covmean)

 missing cuda symbols while dynamic loading
 cuFile initialization failed


In [5]:
def subtract_mean(x: torch.Tensor) -> torch.Tensor:
    mean = [0.406, 0.456, 0.485]
    x[:, 0, :, :] -= mean[0]
    x[:, 1, :, :] -= mean[1]
    x[:, 2, :, :] -= mean[2]
    return x

def normalize_tensor(x: torch.Tensor, eps: float = 1e-10) -> torch.Tensor:
    norm_factor = torch.sqrt(torch.sum(x**2, dim=1, keepdim=True))
    return x / (norm_factor + eps)

def spatial_average(x: torch.Tensor, keepdim: bool = True) -> torch.Tensor:
    return x.mean([2, 3], keepdim=keepdim)

def get_features(image, radnet):

    # If input has just 1 channel, repeat channel to have 3 channels
    if image.shape[1]:
        image = image.repeat(1, 3, 1, 1)

    # Change order from 'RGB' to 'BGR'
    image = image[:, [2, 1, 0], ...]

    # Subtract mean used during training
    image = subtract_mean(image)

    # Get model outputs
    with torch.no_grad():
        feature_image = radnet.forward(image)
        # flattens the image spatially
        feature_image = spatial_average(feature_image, keepdim=False)

    return feature_image

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

Using cuda


In [7]:
# Load model that will be used totransform images into features
radnet = torch.hub.load("Warvito/radimagenet-models", model="radimagenet_resnet50", verbose=True)
radnet.to(device)
radnet.eval()

Using cache found in /home/jdafflon/.cache/torch/hub/Warvito_radimagenet-models_main


ResNet50(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1.001e-05, momentum=0.01, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm

## Experiment 1 - CIFAR

- Use some CIFAR data and compute the FID

In [8]:
# get the dataset 
dl      = DataLoader(torchvision.datasets.CIFAR10('data/cifar', train=True, download=True))
x_train = dl.dataset.data

bs = 2048
images1 = torch.from_numpy(x_train[:bs].astype(np.float32)       / 255 )
images2 = torch.from_numpy(x_train[bs:2*bs].astype(np.float32)   / 255 )

# re-shape images (channel-first) to be on the same size as what MONAI expects
images1 = torch.moveaxis(images1, 3, 1)
images2 = torch.moveaxis(images2, 3, 1)

images1.shape

Files already downloaded and verified


torch.Size([2048, 3, 32, 32])

In [9]:
# Transform the first set of images into features
real_eval_feats = []
features_real = get_features(images1.to(device), radnet)
real_eval_feats.append(features_real.cpu())
real_eval_feats = torch.cat(real_eval_feats, axis=0)

# Transform the second set of images into features
synth_eval_feats = []
features_synth = get_features(images2.to(device), radnet)
synth_eval_feats.append(features_synth.cpu())
synth_eval_feats = torch.cat(synth_eval_feats, axis=0)

print(real_eval_feats.shape, synth_eval_feats.shape)

torch.Size([2048, 2048]) torch.Size([2048, 2048])


### FID

In [10]:
# Fast-FID
fid = FIDMetric()
t0 = time()
res = fid(synth_eval_feats.to(device), real_eval_feats.to(device), fid_type='fast_fid')
t_fast_fid = time() - t0
print(f'Fast-fid : {res}')
print(f'Time: {t_fast_fid}')
print('')

# FID (according to PIQ implementation)
fid = FIDMetric()
t0 = time()
res = fid(synth_eval_feats.to(device), real_eval_feats.to(device), fid_type='fid')
t_fid = time() - t0
print(f'Fid : {res}')
print(f'Time: {t_fid}')
print('')

# FID - piq (pip installed)
res = piq.FID()(synth_eval_feats, real_eval_feats)
print(f'Fid (PIQ - pip installed) : {res}')
t_piq = time() - t0
print(f'Time: {t_piq}')
print('')

# numpy FID
fid = FIDMetric()
res = fid(synth_eval_feats.to(device), real_eval_feats.to(device), fid_type='numpy_fid')
print(f'Fid (numpy): {res}')
t_numpy = time() - t0
print(f'Time: {t_numpy}')
print('')

# Pytorch FID
fid = FIDMetric()
res = fid(synth_eval_feats.to(device), real_eval_feats.to(device), fid_type='pytorch_fid')
print(f'Fid (pytorch) : {res}')
t_pytorch = time() - t0
print(f'Time: {t_pytorch}')
print('')

Fast-fid : 1.0495033123765154
Time: 29.042437314987183

Fid : 5.4749615860228005
Time: 3.2783079147338867

Fid (PIQ - pip installed) : 5.47496158602668
Time: 5.4626429080963135

Fid (numpy): 1.049605931079455
Time: 12.708997249603271

Fid (pytorch) : 1.049605931079455
Time: 19.505873203277588



## Experiment 2: increase the batch size


In [11]:
bs = 10000
images1 = torch.from_numpy(x_train[:bs].astype(np.float32)       / 255 )
images2 = torch.from_numpy(x_train[bs:2*bs].astype(np.float32)   / 255 )

# re-shape images (channel-first) to be on the same size as what MONAI expects
images1 = torch.moveaxis(images1, 3, 1)
images2 = torch.moveaxis(images2, 3, 1)

images1.shape

# Transform the first set of images into features
real_eval_feats = []
features_real = get_features(images1.to(device), radnet)
real_eval_feats.append(features_real.cpu())
real_eval_feats = torch.cat(real_eval_feats, axis=0)


# Transform the second set of images into features
synth_eval_feats = []
features_synth = get_features(images2.to(device), radnet)
synth_eval_feats.append(features_synth.cpu())
synth_eval_feats = torch.cat(synth_eval_feats, axis=0)

print(real_eval_feats.shape, synth_eval_feats.shape)

torch.Size([10000, 2048]) torch.Size([10000, 2048])


In [12]:
# Fast-FID
# should only be used when the number of samples is smaller or equal than the latent space
# throws an error
#fid = FIDMetric()
#t0 = time()
#res = fid(synth_eval_feats.to(device), real_eval_feats.to(device), fid_type='fast_fid')
#t_fast_fid = time() - t0
#print(f'Fast-fid : {res}')
#print(f'Time: {t_fast_fid}')
#print('')

# FID (according to PIQ implementation)
fid = FIDMetric()
t0 = time()
res = fid(synth_eval_feats.to(device), real_eval_feats.to(device), fid_type='fid')
t_fid = time() - t0
print(f'Fid : {res}')
print(f'Time: {t_fid}')
print('')

# FID - piq (pip installed)
res = piq.FID()(synth_eval_feats, real_eval_feats)
print(f'Fid (PIQ - pip installed) : {res}')
t_piq = time() - t0
print(f'Time: {t_piq}')
print('')

# numpy FID
fid = FIDMetric()
res = fid(synth_eval_feats.to(device), real_eval_feats.to(device), fid_type='numpy_fid')
print(f'Fid (numpy): {res}')
t_numpy = time() - t0
print(f'Time: {t_numpy}')
print('')

# Pytorch FID
fid = FIDMetric()
res = fid(synth_eval_feats.to(device), real_eval_feats.to(device), fid_type='pytorch_fid')
print(f'Fid (pytorch) : {res}')
t_pytorch = time() - t0
print(f'Time: {t_pytorch}')
print('')

Fid : 4.973012319066413
Time: 3.657036781311035

Fid (PIQ - pip installed) : 4.9730123190646935
Time: 5.928958892822266

Fid (numpy): 0.23648015908705133
Time: 20.966081857681274

Fid (pytorch) : 0.23648015908705133
Time: 32.78556966781616



## Example 3: Generate some random features  a small sample size

In [13]:
# Code from github.com/mseitzer/pytorch-fid
from pytorch_fid.fid_score import calculate_frechet_distance

dist1_np = np.random.normal(150, 8.0, size=(100, 2048))
dist2_np = np.random.normal(150, 8.0, size=(100, 2048))

dist1_pt = torch.tensor(dist1_np)
dist2_pt = torch.tensor(dist2_np)

dist1_np_mu = np.mean(dist1_np, axis=0)
dist1_np_sigma = np.cov(dist1_np, rowvar=False)

dist2_np_mu = np.mean(dist2_np, axis=0)
dist2_np_sigma = np.cov(dist2_np, rowvar=False)


In [14]:
dist1_np.shape

(100, 2048)

In [15]:
# Fast-FID
fid = FIDMetric()
t0 = time()
res = fid(dist1_pt.to(device), dist2_pt.to(device), fid_type='fast_fid')
t_fast_fid = time() - t0
print(f'Fast-fid : {res}')
print(f'Time: {t_fast_fid}')
print('')

# FID (according to PIQ implementation)
fid = FIDMetric()
t0 = time()
res = fid(dist1_pt.to(device), dist2_pt.to(device), fid_type='fid')
t_fid = time() - t0
print(f'Fid : {res}')
print(f'Time: {t_fid}')
print('')

# FID - piq (pip installed)
piq_output = piq.FID()(dist1_pt, dist2_pt)
print(f'Fid (PIQ - pip installed) : {piq_output}')
t_piq = time() - t0
print(f'Time: {t_piq}')
print('')

# numpy FID
fid = FIDMetric()
res = fid(dist1_pt.to(device), dist2_pt.to(device), fid_type='numpy_fid')
print(f'Fid (numpy): {res}')
t_numpy = time() - t0
print(f'Time: {t_numpy}')
print('')

# Pytorch FID
fid = FIDMetric()
res = fid(dist1_pt.to(device), dist2_pt.to(device), fid_type='pytorch_fid')
print(f'Fid (pytorch) : {res}')
t_pytorch = time() - t0
print(f'Time: {t_pytorch}')
print('')

# Pytorch FID
res = calculate_frechet_distance(dist1_np_mu, dist1_np_sigma, dist2_np_mu, dist2_np_sigma)
print(f'Fid (pytorch - pip installed) : {res}')
t_pytorch = time() - t0
print(f'Time: {t_pytorch}')
print('')


Fast-fid : 216228.1563925882
Time: 0.02732253074645996

Fid : 216228.18513239708
Time: 4.736844062805176

Fid (PIQ - pip installed) : 216228.1851328744
Time: 7.180821895599365

Fid (numpy): 216228.1303018271
Time: 17.778093099594116

Fid (pytorch) : 216228.1303018271
Time: 29.291253089904785

Fid (pytorch - pip installed) : 216228.1303018271
Time: 40.351163148880005



## Exemple 4  -  Generate features with a bigger sample size

In [16]:
dist1_np_3 = np.random.normal(150, 8.0, size=(2048, 2048))
dist2_np_3 = np.random.normal(150, 8.0, size=(2048, 2048))

dist1_pt_3 = torch.tensor(dist1_np_3)
dist2_pt_3 = torch.tensor(dist2_np_3)

dist1_np_mu_3 = np.mean(dist1_np_3, axis=0)
dist1_np_sigma_3 = np.cov(dist1_np_3, rowvar=False)

dist2_np_mu_3 = np.mean(dist2_np_3, axis=0)
dist2_np_sigma_3 = np.cov(dist2_np_3, rowvar=False)


In [17]:
dist1_np_3.shape

(2048, 2048)

In [18]:
# Fast-FID
fid = FIDMetric()
t0 = time()
res = fid(dist1_pt_3.to(device), dist2_pt_3.to(device), fid_type='fast_fid')
t_fast_fid = time() - t0
print(f'Fast-fid : {res}')
print(f'Time: {t_fast_fid}')
print('')

# FID (according to PIQ implementation)
fid = FIDMetric()
t0 = time()
res = fid(dist1_pt_3.to(device), dist2_pt_3.to(device), fid_type='fid')
t_fid = time() - t0
print(f'Fid : {res}')
print(f'Time: {t_fid}')
print('')

# FID - piq (pip installed)
piq_output = piq.FID()(dist1_pt_3, dist2_pt_3)
print(f'Fid (PIQ - pip installed) : {piq_output}')
t_piq = time() - t0
print(f'Time: {t_piq}')
print('')

# numpy FID
fid = FIDMetric()
res = fid(dist1_pt_3.to(device), dist2_pt_3.to(device), fid_type='numpy_fid')
print(f'Fid (numpy): {res}')
t_numpy = time() - t0
print(f'Time: {t_numpy}')
print('')

# Pytorch FID
fid = FIDMetric()
res = fid(dist1_pt_3.to(device), dist2_pt_3.to(device), fid_type='pytorch_fid')
print(f'Fid (pytorch) : {res}')
t_pytorch = time() - t0
print(f'Time: {t_pytorch}')
print('')

# Pytorch FID
res = calculate_frechet_distance(dist1_np_mu_3, dist1_np_sigma_3, dist2_np_mu_3, dist2_np_sigma_3)
print(f'Fid (pytorch - pip installed) : {res}')
t_pytorch = time() - t0
print(f'Time: {t_pytorch}')
print('')


Fast-fid : 65708.97426655755
Time: 11.922155380249023

Fid : 65721.08186852976
Time: 3.6585769653320312

Fid (PIQ - pip installed) : 65721.08186852976
Time: 5.559947967529297

Fid (numpy): 65708.9742586891
Time: 17.802000999450684

Fid (pytorch) : 65708.9742586891
Time: 29.01366424560547

Fid (pytorch - pip installed) : 65708.9742586891
Time: 38.54029583930969



## Example 5 - Compute features and FIQ from random values

In [19]:
dist1_np_4 = np.random.normal(150, 8.0, size=(100, 1, 64, 64)).astype(np.float32)
dist2_np_4 = np.random.normal(150, 8.0, size=(100, 1, 64, 64)).astype(np.float32)
dist1_pt_4 = torch.tensor(dist1_np_4)
dist2_pt_4 = torch.tensor(dist2_np_4)

# Transform the first set of images into features
real_eval_feats_4 = []
features_real_4 = get_features(dist1_pt_4.to(device), radnet)
real_eval_feats_4.append(features_real_4.cpu())
real_eval_feats_4 = torch.cat(real_eval_feats_4, axis=0)

# Transform the second set of images into features
synth_eval_feats_4 = []
features_synth_4 = get_features(dist2_pt_4.to(device), radnet)
synth_eval_feats_4.append(features_synth_4.cpu())
synth_eval_feats_4 = torch.cat(synth_eval_feats_4, axis=0)

In [20]:
print(real_eval_feats_4.shape, synth_eval_feats_4.shape)

torch.Size([100, 2048]) torch.Size([100, 2048])


In [21]:
# Fast-FID
fid = FIDMetric()
t0 = time()
res = fid(synth_eval_feats_4.to(device), real_eval_feats_4.to(device), fid_type='fast_fid')
t_fast_fid = time() - t0
print(f'Fast-fid : {res}')
print(f'Time: {t_fast_fid}')
print('')

# FID (according to PIQ implementation)
fid = FIDMetric()
t0 = time()
res = fid(synth_eval_feats_4.to(device), real_eval_feats_4.to(device), fid_type='fid')
t_fid = time() - t0
print(f'Fid : {res}')
print(f'Time: {t_fid}')
print('')

# FID - piq (pip installed)
piq_output = piq.FID()(synth_eval_feats_4, real_eval_feats_4)
print(f'Fid (PIQ - pip installed) : {piq_output}')
t_piq = time() - t0
print(f'Time: {t_piq}')
print('')

# numpy FID
fid = FIDMetric()
res = fid(synth_eval_feats_4.to(device), real_eval_feats_4.to(device), fid_type='numpy_fid')
print(f'Fid (numpy): {res}')
t_numpy = time() - t0
print(f'Time: {t_numpy}')
print('')

# Pytorch FID
fid = FIDMetric()
res = fid(synth_eval_feats_4.to(device), real_eval_feats_4.to(device), fid_type='pytorch_fid')
print(f'Fid (pytorch) : {res}')
t_pytorch = time() - t0
print(f'Time: {t_pytorch}')
print('')


Fast-fid : 32.00129289074448
Time: 0.04369330406188965

Fid : 34.35554812299779
Time: 0.2589125633239746

Fid (PIQ - pip installed) : 34.35554812299915
Time: 0.41750264167785645

Fid (numpy): 32.00137515639551
Time: 4.0166075229644775

Fid (pytorch) : 32.00137515639551
Time: 8.238928079605103

